In [2]:
test_input = """Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."""

In [3]:
import re
import numpy as np
from functools import cache, lru_cache
import math

In [4]:
reg = ".*Each ore robot costs (\d+) ore\. Each clay robot costs (\d+) ore\. Each obsidian robot costs (\d+) ore and (\d+) clay\. Each geode robot costs (\d+) ore and (\d+) obsidian\."

In [5]:
def add(a, b):
    return tuple(map(lambda x, y: x + y, a, b))

def sub(a, b):
    return tuple(map(lambda x, y: x - y, a, b))

In [6]:
blueprints = []

for l in test_input.split("\n"):
# for l in open("inputs/19").read().split("\n"):
    print(l)
    m = re.match(reg, l)
    costs = list(map(int, m.groups()))

    # ore cost, clay cost, obsidian cost, geode cost
    ore_robot_costs = (costs[0], 0, 0, 0)
    clay_robot_costs = (costs[1], 0, 0, 0)
    obsidian_robot_costs = (costs[2], costs[3], 0, 0)
    geode_robot_costs = (costs[4], 0, costs[5], 0)

    blueprint = (ore_robot_costs, clay_robot_costs, obsidian_robot_costs, geode_robot_costs)
    blueprints.append(blueprint)

Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.


In [7]:
blueprints

[((4, 0, 0, 0), (2, 0, 0, 0), (3, 14, 0, 0), (2, 0, 7, 0)),
 ((2, 0, 0, 0), (3, 0, 0, 0), (3, 8, 0, 0), (3, 0, 12, 0))]

In [8]:
TIME_LIMIT = 24

In [9]:
START_ORE = (0, 0, 0, 0)
START_ROBOTS = (1, 0, 0, 0)

In [10]:
def max_out_resources(t, max_prices, resources):
    theoretical_maxes = [t*m for m in max_prices]
    # never have enough geodes
    theoretical_maxes[-1] = math.inf

    return [r >= tm for r, tm in zip(resources, theoretical_maxes)]
    # return tuple([r if r <= tm else math.inf for r, tm in zip(resources, theoretical_maxes)])

In [11]:
def apply_mask(resources, mask):
    return tuple([r if not m else math.inf for r, m in zip(resources, mask)])

In [12]:
def total_geodes(blueprint, time):
    max_prices = [max(b[i] for b in blueprint) for i in range(4)]

    # @cache
    @lru_cache(maxsize=10_000_000)
    def f(time, blueprint, ore_resources, robot_resources):
        if time == 0:
            return ore_resources[-1]

        mask = max_out_resources(time-1, max_prices, ore_resources)

        after_building_resources = [sub(ore_resources, b) for b in blueprint]
        building_feasibilities = [all(a >= 0 for a in r) for r in after_building_resources]

        all_feasible = all(building_feasibilities)

        new_ore_resources = add(ore_resources, robot_resources)

        # don't build. just collect
        # NEVER do this if it's possible to build everything. why would you?
        if not all_feasible:
            max_geodes = f(time-1, blueprint, apply_mask(new_ore_resources, mask), apply_mask(robot_resources, mask))
        else:
            max_geodes = 0

        for i, (feasible, after_building) in enumerate(zip(building_feasibilities, after_building_resources)):
            if not feasible:
                continue

            # don't need to build something maxed out
            if ore_resources[i] == math.inf:
                continue

            new_robots = list(robot_resources)
            new_robots[i] += 1
            new_robots = apply_mask(new_robots, mask)

            new_ore_resources = apply_mask(add(after_building, robot_resources), mask)

            m = f(time-1, blueprint, new_ore_resources, new_robots)
            
            if m > max_geodes:
                max_geodes = m

        # if max_geodes > 0:
        #     print(time, ore_resources, robot_resources, max_geodes)

        return max_geodes
    
    return f(time, blueprint, START_ORE, START_ROBOTS)

In [13]:
print(blueprints[1])

((2, 0, 0, 0), (3, 0, 0, 0), (3, 8, 0, 0), (3, 0, 12, 0))


In [15]:
total_geodes(blueprints[1], 19)

1

In [89]:
# total_geodes.cache_info()

In [90]:
s = 0

for (i, b) in enumerate(blueprints):
    print(i)
    r = total_geodes(b, TIME_LIMIT)
    print(r)
    s += (i+1) * r
    print()

0
9

1
12



In [91]:
s

33

In [92]:
prod = 1

for (i, b) in enumerate(blueprints[:3]):
    print(i)

    p = (i+1) * total_geodes(b, 32)
    # p = 1
    prod *= p

0


In [ ]:
prod

1